In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel("../private_data/survey_results_aggregated/data.xlsx")

In [ ]:
# country cleaning
data['job_country'] = data['job_country'].str.strip()
data['search_country'] = data['search_country'].str.strip()

country_dict = {
  "España, Presencial en Sevilla": "Spain",
  "españa": "Spain",
  "Paris,France": "France",
  "germany": "Germany",
  "United Kingdom": "UK",
  "The Netherlands": "Netherlands",
  "Netherlands, but also applying to jobs in the UK and Europe": "Netherlands",
  "Emea": "Other",
  "UK, Ireland, France, Spain": "Other",
  "Switzerland and France": "Other",
  "Europe": "Other",
}
data['job_country'] = data['job_country'].replace(country_dict)
data['search_country'] = data['search_country'].replace(country_dict)

In [ ]:
# data.groupby(["search_country"]).agg({"search_country": "count"})

In [ ]:
# learned techs cleaning
data.columns

In [ ]:
students_with_dev_job = data[
  (data["post_bootcamp_situation"] == "I found a Full-Time job as a developer (or a similar role)") |
  (data["post_bootcamp_situation"] == "I found a Part-Time job as a developer (or a similar role)") |
  (data["post_bootcamp_situation"] == "I found work as a Freelance developer, with a relevant source of income")
]

students_with_not_dev_job = data[
  data["post_bootcamp_situation"] == "I found a job, but is not related to the bootcamp"
]

students_with_no_job = data[
  data["post_bootcamp_situation"] == "I haven't found a job."
]

In [ ]:
students_with_dev_job.count()

In [ ]:
students_with_not_dev_job.count()

In [ ]:
students_with_no_job.count()

In [ ]:
# No se notan patrones en las technologias de estudiantes con trabajo, sin trabajo o trabajo no relacionado.